In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Filtramos los warnings
import warnings
warnings.filterwarnings("ignore")

# Forzar uso CPU en caso de dispones GPU en el PC
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


# 4.1.- Juego Lunar Lander de Open AI GYM (con Stable Baseline)


* GYM de Open AI https://gym.openai.com/: Gym es un conjunto de herramientas para desarrollar y comparar algoritmos de Aprendizaje por Refuerzo (RL). Esta librería ofrece diferentes juegos (entornos) como los juegos de Atari en los que poder probar los algoritmos de RL.


* Stable Baseline https://stable-baselines.readthedocs.io/: Librería en python en la que tiene implementados diferentes algoritmos y funcionalidades de Aprendizaje por Refuerzo.


<hr>

## 1.- Instalación de las librerías (en un Python 3.7)

In [ ]:
!pip3 install tensorflow==1.15.0
!pip3 install stable_baselines
!pip3 install box2d-py
!pip3 install gym[atari]==0.19.0

<hr>

## 2.- Importamos las librerías de GYM y Stable Baselines


In [3]:
import gym 

from stable_baselines import ACER
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.evaluation import evaluate_policy


# Definición de constantes
ENVIRONMENT_NAME = 'LunarLander-v2'
FILE_MODEL_NAME = "ACER_LunarLander_Model"


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



<hr>

## 3.- Instanciamos el entorno del "Lunar Lander" y realizamos acciones aleatorias


* Lunar Lander es un sencillo juego que consiste en aterrizar una nave entre dos banderas.

* En este juego se pueden realizar 4 ***acciones*** que son:

    + no hacer nada            (acción 0)
    + activar motor izquierdo  (acción 1)
    + activar motor de abajo   (acción 2)
    + activar motor derecho    (acción 3)
    

* Las ***observaciones*** estan definidas por ***8 estados***: las coordenadas del módulo de aterrizaje en x & y, su lineal velocidades en x & y, su ángulo, su velocidad angular y dos valores booleanos que representan si cada pierna está en contacto con el suelo o no.


* ***Estado inicial***: La nave comienza en la parte superior central de la pantalla con una fuerza inicial aleatoria aplicada a su centro de masa.


* ***Recompensas***: 

    + Recompensa por moverse desde la parte superior de la pantalla a la plataforma de aterrizaje y venir para descansar es de unos 100-140 puntos.
    + Si el módulo de aterrizaje se aleja de la plataforma de aterrizaje, pierde la recompensa.
    + Si el módulo de aterrizaje se estrella, recibe -100 puntos adicionales. 
    + Cada pata de la nave en contacto con el suelo es +10 puntos.
    + Activar el motor principal es -0.3 puntos. 
    + Activar el motor lateral es -0.03 puntos. 
    + Resuelto son 200 puntos.
    
    
* ***Fin del episodio***: La partida termina en alguno de los 3 casos:

    + la nave se estrella (el cuerpo de la nave entra en contacto con la luna).
    + la nave sale de la ventana gráfica.
    + la nave se queda quieta (no se mueve)


* Código: https://github.com/openai/gym/blob/master/gym/envs/box2d/lunar_lander.py


In [4]:
env = gym.make(ENVIRONMENT_NAME)

print('Acciones del Juego: {}'.format(env.action_space.n))

episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()  # Inicializamos el entorno
    done = False         # Flag de finalización de la partida (episodio)
    score = 0            # Contador de recompensas
    
    # ! A Jugar ¡ (Hasta que termine la partida -> done == True)
    while not done:
        env.render()
        action = env.action_space.sample()              # Seleccionamos una acción Aleatoria
        n_state, reward, done, info = env.step(action)  # Realizamos la acción aleatoria y obtenemos: 
                                                        # 1. Lista de estados, 2. Recompensa, 3. ¿Fin del juego?, 4. info
        score+=reward                                   # Sumamos la recompensa de la acción
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Acciones del Juego: 4
Episode:1 Score:-225.53990591703925
Episode:2 Score:-102.87568725927923
Episode:3 Score:-79.17159308892002
Episode:4 Score:-86.45601192382098
Episode:5 Score:-226.69039410632342


<hr>

## 4.- Entrenamos un agente que aprenda a aterrizar la nave entre las banderas y exportamos el modelo

In [5]:
# Creacción del entorno
env = gym.make(ENVIRONMENT_NAME)

# Instancia del Algoritmo de aprendizaje
model = ACER(MlpPolicy, env, verbose=1)

# Entrenamiento del Agente, indicando el número de acciones que tiene que realizar (en varios episodios)
model.learn(total_timesteps=100000)

# Obtenemos información del entrenamiento del agente con la función evaluate_policy(), indicando el numero de episodios de evaluación
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5)
print("Reward:{:2f} +/- {:2f}".format(mean_reward, std_reward))

# Exportación del Agente Entrenado
model.save(FILE_MODEL_NAME)

Wrapping the env in a DummyVecEnv.




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


----------------------------------
| avg_norm_adj        | 6.63     |
| avg_norm_g          | 27.5     |
| avg_norm_grads_f    | 23.8     |
| avg_norm_k          | 2        |
| avg_norm_k_dot_g    | 27.5     |
| entropy             | 29.1     |
| explained_variance  | 7.21e-06 |
| fps                 | 0        |
| loss                | 3.88     |
| loss_bc             | -0       |
| loss_f              | -9.54    |
| loss_policy         | -9.54    |
|

----------------------------------
| avg_norm_adj        | 0.0224   |
| avg_norm_g          | 25.8     |
| avg_norm_grads_f    | 25.7     |
| avg_norm_k          | 2.63     |
| avg_norm_k_dot_g    | 36.5     |
| entropy             | 15.2     |
| explained_variance  | -0.0874  |
| fps                 | 837      |
| loss                | 12.2     |
| loss_bc             | -0       |
| loss_f              | 3.98     |
| loss_policy         | 3.98     |
| loss_q              | 16.8     |
| mean_episode_length | 173      |
| mean_episode_reward | -263     |
| norm_grads          | 14.6     |
| norm_grads_policy   | 1.21     |
| norm_grads_q        | 14.5     |
| total_timesteps     | 10020    |
----------------------------------
----------------------------------
| avg_norm_adj        | 1.99     |
| avg_norm_g          | 24.2     |
| avg_norm_grads_f    | 23.2     |
| avg_norm_k          | 1.77     |
| avg_norm_k_dot_g    | 21.7     |
| entropy             | 23       |
| explained_variance

----------------------------------
| avg_norm_adj        | 35.4     |
| avg_norm_g          | 185      |
| avg_norm_grads_f    | 167      |
| avg_norm_k          | 2.35     |
| avg_norm_k_dot_g    | 178      |
| entropy             | 14.1     |
| explained_variance  | -0.585   |
| fps                 | 698      |
| loss                | 1.55e+03 |
| loss_bc             | -0       |
| loss_f              | -45.1    |
| loss_policy         | -45.1    |
| loss_q              | 3.19e+03 |
| mean_episode_length | 455      |
| mean_episode_reward | 5.31     |
| norm_grads          | 1.3e+03  |
| norm_grads_policy   | 148      |
| norm_grads_q        | 1.29e+03 |
| total_timesteps     | 34020    |
----------------------------------
----------------------------------
| avg_norm_adj        | 0        |
| avg_norm_g          | 86.1     |
| avg_norm_grads_f    | 86.1     |
| avg_norm_k          | 1.85     |
| avg_norm_k_dot_g    | 79.7     |
| entropy             | 11.5     |
| explained_variance

----------------------------------
| avg_norm_adj        | 39.3     |
| avg_norm_g          | 150      |
| avg_norm_grads_f    | 130      |
| avg_norm_k          | 1.68     |
| avg_norm_k_dot_g    | 142      |
| entropy             | 9.4      |
| explained_variance  | -2.15    |
| fps                 | 706      |
| loss                | 237      |
| loss_bc             | -0       |
| loss_f              | -26.8    |
| loss_policy         | -26.8    |
| loss_q              | 527      |
| mean_episode_length | 404      |
| mean_episode_reward | 71.2     |
| norm_grads          | 455      |
| norm_grads_policy   | 28.1     |
| norm_grads_q        | 454      |
| total_timesteps     | 58020    |
----------------------------------
----------------------------------
| avg_norm_adj        | 1.39     |
| avg_norm_g          | 8.12     |
| avg_norm_grads_f    | 7.21     |
| avg_norm_k          | 2.42     |
| avg_norm_k_dot_g    | 9.09     |
| entropy             | 14.6     |
| explained_variance

----------------------------------
| avg_norm_adj        | 1.1      |
| avg_norm_g          | 6.63     |
| avg_norm_grads_f    | 5.92     |
| avg_norm_k          | 2.25     |
| avg_norm_k_dot_g    | 7.07     |
| entropy             | 21.4     |
| explained_variance  | 0.348    |
| fps                 | 720      |
| loss                | 0.279    |
| loss_bc             | -0       |
| loss_f              | -1.23    |
| loss_policy         | -1.23    |
| loss_q              | 3.45     |
| mean_episode_length | 252      |
| mean_episode_reward | -4.58    |
| norm_grads          | 35.5     |
| norm_grads_policy   | 3.29     |
| norm_grads_q        | 35.3     |
| total_timesteps     | 82020    |
----------------------------------
----------------------------------
| avg_norm_adj        | 11.9     |
| avg_norm_g          | 53.6     |
| avg_norm_grads_f    | 50       |
| avg_norm_k          | 1.65     |
| avg_norm_k_dot_g    | 29.2     |
| entropy             | 18       |
| explained_variance

<hr>

## 5.- Importamos el modelo y lo ponemos a jugar

In [7]:
# Importamos el modelo ya entrenado
trained_model = ACER.load(FILE_MODEL_NAME)

# Instanciamos el entorno del juego Lunar Lander
env = gym.make(ENVIRONMENT_NAME)

# Vaciamos la memoria de las observaciones (Lista de estados)
obs = env.reset()

# ! A JUGAR ¡ (mientras no se termine la partida)
done = False
score = 0 
steps = 0
while not done:
    action, _states = trained_model.predict(obs) # Dada una observación, pedimos al agente la acción a realizar
    obs, rewards, done, info = env.step(action)  # Realizamos la acción y nos devuelve:
                                                 # 1. Lista de estados, 2. Recompensa, 3. ¿Fin del juego?, 4. info
    score += rewards  # Sumamos la recompensa
    steps +=1         # Sumamos los pasos
    env.render()
    
print('Score Game:{}'.format(score))
print('Steps Game:{}'.format(steps))

# Cerramos el entorno
env.close()

Loading a model without an environment, this model cannot be trained until it has a valid environment.
Score Game:15.385338285620875
Steps Game:193
